In [33]:
# !pip install pandas
# ! pip install tensorly

In [45]:
import torch
import torch.nn as nn
import tensorly as tl
from tensorly.decomposition import tucker
from tqdm import tqdm
# from tensorly.metrics import norm


In [46]:
tl.set_backend('pytorch')

In [47]:

class IdentityModelPlusNoise(nn.Module):
    """
    A PyTorch model that returns its input unchanged.
    
    The model is designed to work with input tensors of shape 
    (batch_size, 32, 32, 32, 1), but it can technically accept any tensor shape.
    
    Parameters:
    - None
    
    Methods:
    - forward(x): Passes the input tensor `x` through the model unchanged.
    """
    
    def __init__(self):
        super(IdentityModelPlusNoise, self).__init__()
    
    def forward(self, x):
        """
        The forward pass of the model.
        
        Parameters:
        - x (torch.Tensor): The input tensor with shape (batch_size, 32, 32, 32, 1)
        
        Returns:
        - torch.Tensor: The unchanged input tensor.
        """
        noise = torch.randn_like(x)
        return x + noise


In [48]:
# Instantiate the model
model = IdentityModelPlusNoise()
batch_size = 10 
# Create a dummy input tensor of the specified shape and type
input_tensor = torch.randn((batch_size, 32, 32, 32, 1), dtype=torch.float32)

# Pass the input tensor through the model
output_tensor = model(input_tensor)

# Check if the output is the same as the input
print(torch.equal(input_tensor, output_tensor))


False


In [49]:

class TuckerDecompositionLoss(torch.nn.Module):
    """
    A custom loss function based on the Tucker decomposition of tensors.
    
    The loss is calculated as the Frobenius norm of the difference between the
    core tensors of the Tucker decomposition of the predicted and target tensors.
    """
    
    def __init__(self):
        super(TuckerDecompositionLoss, self).__init__()

    def forward(self, prediction, target):
        # Perform Tucker decomposition on both tensors
        loss = 0.0
        for i in tqdm(range(len(prediction))):

            core_pred, _ = tucker(prediction[i], rank=[5, 5, 5, 5])  # Adjust ranks as needed
            core_target, _ = tucker(target[i], rank=[5, 5, 5, 5])  # Adjust ranks as needed
        
        # Compute the loss as the Frobenius norm between the core tensors
            loss += torch.norm(core_pred - core_target, 2)
        loss_m = torch.mean(loss) 
        return loss_m, core_pred, core_target

In [50]:
output_tensor.shape

torch.Size([10, 32, 32, 32, 1])

In [51]:
# Assume model, optimizer, data_loader are predefined
loss_function = TuckerDecompositionLoss()

# Compute loss
loss, core_pred, core_target = loss_function(output_tensor, input_tensor)



100%|██████████| 10/10 [00:36<00:00,  3.63s/it]


In [53]:
core_pred.shape, core_target.shape

(torch.Size([5, 5, 5, 1]), torch.Size([5, 5, 5, 1]))

In [54]:
loss,

(tensor(610.4222),)

In [ ]:
# Install the necessary packages

import gymnasium as gym
import torch
import torch.nn as nn
from transformers import GPT2Model, GPT2Config, GPT2Tokenizer, GPT2LMHeadModel
from collections import deque
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
from torch.optim import AdamW


In [ ]:
# Load pre-trained model and tokenizer
model_name = "gpt2"  # You can replace this with other model names
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)


In [ ]:
# Prepare input text
input_text = "Where is China?"
input_ids = tokenizer.encode(input_text, return_tensors='pt').to('cuda')
model = model.to('cuda')

In [ ]:
# Generate the attention mask (1 for input tokens, 0 for padding)
attention_mask = torch.ones(input_ids.shape, dtype=torch.long).to('cuda')  # Ensure the mask is on the same device as the model

# Set pad_token_id explicitly if it's not defined
if model.config.pad_token_id is None:
    model.config.pad_token_id = model.config.eos_token_id

# Generate text with the attention_mask and pad_token_id settings
output = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=50,
    pad_token_id=model.config.pad_token_id
)

# Decoding the output to human-readable text (optional)
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded_output)

In [ ]:
torch.cuda.memory_allocated()

In [ ]:
# Sample dataset (a list of sentences)
# EDA-themed dataset
eda_dataset = [
    "Electronic Design Automation involves complex software algorithms.",
    "EDA tools are essential for designing integrated circuits.",
    "In EDA, simulation verifies the functionality of a chip design.",
    "Layout optimization is a crucial step in chip design.",
    "EDA software assists engineers in managing power consumption and heat dissipation.",
    "Routing and placement are key challenges in EDA.",
    "Machine learning can enhance the automation process in EDA.",
    "EDA tools are evolving with advancements in semiconductor technology.",
    "Design rule checking is vital to ensure manufacturing feasibility.",
    "EDA is critical for the development of efficient and compact electronic devices."
    "Deep likes eating paneer.",
    "Nirjhor lives in Raleigh", 
]

# Tokenize sentences
# Set the pad token to the EOS token
tokenizer.pad_token = tokenizer.eos_token

inputs = tokenizer(eda_dataset, padding=True, truncation=True, return_tensors="pt")
inputs['labels'] = inputs.input_ids.detach().clone()


# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Move model to GPU if available
model.to(device)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
model.train()
for epoch in tqdm(range(100)):  # number of epochs
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(**inputs.to(device))
    loss = outputs.loss

    # Backward pass
    loss.backward()
    optimizer.step()

    # print(f"Epoch {epoch}, Loss: {loss.item()}")



In [ ]:
# Assuming you have a test dataset in the format of prompts and expected completions
test_dataset = [
    {"prompt": "the beans are", "expected": ""},
    {"prompt": "key challenges are", "expected": ""},
    {"prompt": "the only way to", "expected": ""},
    {"prompt": "nirjhor lives", "expected": ""}
    
    # Add more test examples
]

# Evaluate the model
for test_case in test_dataset:
    input_ids = tokenizer.encode(test_case["prompt"], return_tensors='pt');
    input_ids = input_ids.to(device);

    # Generate text
    output = model.generate(input_ids, max_length=50);
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True);

    # Compare generated text with expected text
    # Here you can apply BLEU score or simply compare the texts manually
    print(f"Prompt: {test_case['prompt']}")
    print(f"Generated: {generated_text}")
    print(f"Expected: {test_case['expected']}")


In [ ]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total number of trainable parameters: {trainable_params}")
#gpt2 has roughly 124 million parameters

In [ ]:
model.dtype

In [ ]:
model